In [3]:
import tsdb

tsdb.list_available_datasets()  # list all available datasets in TSDB
data = tsdb.load_dataset('physionet_2019')  # select the dataset you need and load it, TSDB will download, extract, and process it automatically
tsdb.download_and_extract('physionet_2019', './ph_19')  # if you need the raw data, use download_and_extract()


'''tsdb.list_cached_data()  # datasets you once loaded are cached, and you can check them with list_cached_data()
tsdb.delete_cached_data(dataset_name='physionet_2019')  # you can delete only one specific dataset and preserve others
tsdb.delete_cached_data()  # or you can delete all cache with delete_cached_data() to free disk space'''

2023-09-16 23:59:20 [INFO]: You're using dataset physionet_2019, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2019
2023-09-16 23:59:20 [INFO]: Start downloading...
2023-09-16 23:59:23 [INFO]: Successfully downloaded data to /var/folders/q6/lk4j1q_132jb5t939ryj_p5r0000gn/T/tmp6ah8p6tf/training_setA.zip.
2023-09-16 23:59:25 [INFO]: Successfully extracted data to /Users/michal/.tsdb_cached_datasets/physionet_2019
2023-09-16 23:59:28 [INFO]: Successfully downloaded data to /var/folders/q6/lk4j1q_132jb5t939ryj_p5r0000gn/T/tmpb7o__rmu/training_setB.zip.
2023-09-16 23:59:30 [INFO]: Successfully extracted data to /Users/michal/.tsdb_cached_datasets/physionet_2019
2023-09-17 00:00:42 [INFO]: Successfully saved to /Users/michal/.tsdb_cached_datasets/physionet_2019/physionet_2019_cache.pkl
2023-09-17 00:00:42 [INFO]: Loaded successfully!
2023-09-17 00:00:42 [INFO]: Start downl

"tsdb.list_cached_data()  # datasets you once loaded are cached, and you can check them with list_cached_data()\ntsdb.delete_cached_data(dataset_name='physionet_2019')  # you can delete only one specific dataset and preserve others\ntsdb.delete_cached_data()  # or you can delete all cache with delete_cached_data() to free disk space"

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score

# Assuming you have already prepared your data (X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor)

class ANN(nn.Module):
    def __init__(self, input_dim):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return self.sig(x)

# Initialize the model
input_dim = 40
model = ANN(input_dim)

# Define the loss function and optimizer
criterion = nn.BCELoss()
initial_lr = 0.001  # Initial learning rate
new_lr = initial_lr * 0.1  # Decrease learning rate
l2_reg = 1e-4  # Adjust regularization strength

optimizer = optim.Adam(model.parameters(), lr=initial_lr)

# Training loop
num_epochs = 50
batch_size = 32

for epoch in range(num_epochs):
    for i in range(0, len(X_train_tensor), batch_size):
        batch_X = X_train_tensor[i:i+batch_size]
        batch_y = y_train_tensor[i:i+batch_size]

        # Check for NaN values in the batch_X (before replacing with 0)
        nan_mask = torch.isnan(batch_X)
        has_nan = torch.any(nan_mask)

        

        if has_nan:
            batch_X[torch.isnan(batch_X)] = 0
            # Reduce learning rate and apply regularization
            
            for param_group in optimizer.param_groups:
                param_group['lr'] = new_lr
            # Apply L2 regularization
            l2_loss = 0.0
            for param in model.parameters():
                l2_loss += torch.norm(param, p=2)

            loss = criterion(model(batch_X), batch_y.view(-1, 1)) + l2_reg * l2_loss
        else:
            # Use the normal learning rate and no regularization
            
            loss = criterion(model(batch_X), batch_y.view(-1, 1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the loss for each epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# Evaluate the model on the test set
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_predictions = (test_outputs > 0.5).float().view(-1)
    accuracy = accuracy_score(y_test_tensor.numpy(), test_predictions.numpy())
    print(f'Test Accuracy: {accuracy}')
